In [ ]:
import langsmith
from langchain import chat_models, prompts, smith
from langchain.schema import output_parser
from langchain.chat_models import AzureChatOpenAI
import os
from dotenv import load_dotenv

load_dotenv()

In [ ]:
llm = AzureChatOpenAI(
    azure_deployment="YYYY",
    openai_api_type="azure", 
    temperature=0.0
)

In [ ]:
from langchain.schema import HumanMessage

message = HumanMessage(
    content="Translate this sentence from English to French. I love programming."
)
llm([message])

In [ ]:
dataset_name = "ds-elderly-sea-70"

In [ ]:
# Filter runs to add to the dataset
client = langsmith.Client()
project_name = "my-chain-example"

runs = client.list_runs(
    project_name= project_name,
    execution_order=1,
    error=False,
)

dataset = client.create_dataset(dataset_name, description="An example dataset")
for run in runs:
    client.create_example(
        inputs=run.inputs,
        outputs=run.outputs,
        dataset_id=dataset.id,
    )

In [ ]:

eval_config = smith.RunEvalConfig(
    evaluators=[
        "cot_qa",
        smith.RunEvalConfig.LabeledCriteria("relevance"),
        smith.RunEvalConfig.LabeledCriteria("coherence"),
        smith.RunEvalConfig.LabeledCriteria("helpfulness"),
        smith.RunEvalConfig.LabeledCriteria("maliciousness"),
        smith.RunEvalConfig.LabeledCriteria("misogyny"),
        smith.RunEvalConfig.Criteria(
            {
                "Useful": "Read the question and check if the answer is useful. At the end can I use the answer to solve my problem?"
                "Respond Y if it is, N if it's not."
            }
        ),
    ],
    custom_evaluators=[],
    eval_llm=llm
)


In [ ]:
import uuid
guid = str(uuid.uuid4())


client = langsmith.Client()
chain_results = client.run_on_dataset(
    dataset_name=dataset_name,
    llm_or_chain_factory=llm,
    evaluation=eval_config,
    project_name=f"test-giving-lord-{guid}",
    concurrency_level=5,
    verbose=True,
)
    

 